
node screenshots?


In [ ]:

const screenshot = require('screenshot-desktop')
const sharp = require('sharp')
//const { Monitor } = require('node-screenshots')

async function screenshotMonitor(monitor) {

  if(!monitor) {
    monitor = 0
  }
  //console.log('Taking screenshot')
  //let monitors = Monitor.all()

  //let capturer = monitors[monitor]
  try {
    let displays = await screenshot.listDisplays()

    let image = await screenshot({ format: 'png', screen: displays[monitor].id })

    console.log(displays[monitor])
    //let rawData = await image.toRaw()

    //console.log('Converting screenshot')
    const imageObj = await sharp(image)
    const metadata = await imageObj.metadata()

    return await imageObj
      .resize(Math.round(metadata.width / 3), Math.round(metadata.height / 3))
      .png()
      .toBuffer()
  } catch (e) {
    console.error(e)
  }
}

module.exports = screenshotMonitor



node window screenshot?


In [ ]:

const fs = require("fs");
const { Window } = require("node-screenshots");


async function screenshotWindow(window) {

  if(!window) {
    window = 0
  }

  let windows = Window.all();
  let item
  if(!Number.isNaN(parseInt(window)))
    item = windows[window]
  else
    item = windows.filter(w => {
      console.log(w.title, w.app_name)
      return w.title.includes(window) || w.app_name && w.app_name.includes(window)
    })[0]

  if(!item)
    return void 0

  /*console.log({
    title: item.title,
    app_name: item.app_name,
    id: item.id,
    x: item.x,
    y: item.y,
    width: item.width,
    height: item.height,
    rotation: item.rotation,
    scaleFactor: item.scaleFactor,
    isPrimary: item.isPrimary,
  })*/

  let image = await windows[window].captureImage()

  return await image.toPng()
}

module.exports = screenshotWindow



get screen size?


In [ ]:

const robot = require("robotjs")

function getScreenSize() {
  return robot.getScreenSize()
}

module.exports = getScreenSize



move mouse?


In [ ]:
const { Monitor } = require("node-screenshots")
const MouseController = require('mouse-controller')
const mc = new MouseController();

function moveMouse(x, y) {
  let monitors = Monitor.all()
  monitors.sort((a, b) => (a.x + a.y) - (b.x + b.y))
  /*console.log({
    x: monitors[0].x,
    y: monitors[0].y,
    width: monitors[0].width,
    height: monitors[0].height,
  })*/
  mc.move(x * monitors[0].width * 3 + monitors[0].x, y * monitors[0].height)
}

module.exports = moveMouse



mouse click?


In [ ]:
const { Monitor } = require("node-screenshots")
const MouseController = require('mouse-controller')
const mc = new MouseController();

function clickMouse(x, y) {
  let monitors = Monitor.all()
  monitors.sort((a, b) => (a.x + a.y) - (b.x + b.y))
  /*console.log({
    x: monitors[0].x,
    y: monitors[0].y,
    width: monitors[0].width,
    height: monitors[0].height,
  })*/
  mc.click(mc.BUTTON.LEFT, {x: x * monitors[0].width * 3 + monitors[0].x, y: y * monitors[0].height})
}

module.exports = clickMouse



send keys?


In [ ]:
// Type "Hello World" then press enter.
var robot = require("robotjs");

const TAP_SPECIAL = [
  'enter',
  'escape',
  'backspace',
  'delete',
]

async function keySend(text, special) {

  // Type "Hello World".
  //robot.typeString(text)

  let modifiers = special.filter(s => !TAP_SPECIAL.includes(s))

  for(let i = 0; i < text.length; i++) {
    let localModifiers = [].concat(modifiers)
    if(text[i].toUpperCase() == text[i] && !localModifiers.includes('shift')) {
      localModifiers.push('shift')
    }
    robot.keyToggle(text[i].toLowerCase(), 'down', modifiers)
    await new Promise(resolve => setTimeout(resolve, 150))
    robot.keyToggle(text[i].toLowerCase(), 'up', modifiers)
  }

  let tap = special.filter(s => TAP_SPECIAL.includes(s))
  if(tap.length > 0) {
    // Press enter.
    robot.keyTap(tap[0], modifiers)
  }

}

module.exports = keySend


mouse control cocoa?


In [ ]:
#include <CoreFoundation/CoreFoundation.h>
#include <CoreGraphics/CoreGraphics.h>

#if __cplusplus
extern "C" {
#endif

typedef enum {
  kButtonLeft = 0,
  kButtonRight = 1,
} Button;

CGEventType previous_event_;
int event_number_ = 32000;

int main(int argc, char *argv[])
{
    previous_event_ = kCGEventNull;
}

void DispatchMouseEvent(CGMouseButton button, CGEventType type, CGPoint position, uint32_t click_count, uint32_t delay)
{
    if (previous_event_ == kCGEventMouseMoved || previous_event_ == kCGEventLeftMouseDragged)
    {
        usleep(delay);
    }

    event_number_++;
    CGEventRef event = CGEventCreateMouseEvent(NULL, type, position, button);
    CGEventSetType(event, type);
    CGEventSetIntegerValueField(event, kCGMouseEventNumber, event_number_);
    CGEventSetIntegerValueField(event, kCGMouseEventClickState, click_count);
    CGEventPost(kCGHIDEventTap, event);
    CFRelease(event);
    previous_event_ = type;
    usleep(delay);
}

CGPoint GetMousePosition()
{
    usleep(200000);
    CGEventRef event = CGEventCreate(NULL);
    CGPoint cursor = CGEventGetLocation(event);
    CFRelease(event);
    return cursor;
}

void SetMousePosition(double x, double y)
{
    CGPoint position = CGPointMake(x, y);
    DispatchMouseEvent(kCGMouseButtonLeft, kCGEventMouseMoved, position, 1, 0);
    previous_event_ = kCGEventMouseMoved;
}

void SetPosition(CGPoint position)
{
    DispatchMouseEvent(kCGMouseButtonLeft, kCGEventMouseMoved, position, 1, 0);
    previous_event_ = kCGEventMouseMoved;
}

void SetButtonState(Button btn, bool is_pressed, CGPoint position, int click_count)
{
    CGEventType type;
    CGMouseButton button;

    switch (btn)
    {
    case kButtonLeft:
        type = (is_pressed ? kCGEventLeftMouseDown : kCGEventLeftMouseUp);
        button = kCGMouseButtonLeft;
        break;

    case kButtonRight:
        type = (is_pressed ? kCGEventRightMouseDown : kCGEventRightMouseUp);
        button = kCGMouseButtonRight;
        break;
    };

    DispatchMouseEvent(button, type, position, click_count, 200000);
}

void MouseClick(Button btn, CGPoint position)
{
    SetButtonState(btn, true, position, 1);  // Press
    SetButtonState(btn, false, position, 1); // Release
}

void Click(Button btn)
{
    CGPoint position = GetMousePosition();
    MouseClick(btn, position);
}

void MouseDoubleClick(Button btn, CGPoint position)
{
    SetButtonState(btn, true, position, 1);
    SetButtonState(btn, false, position, 1);
    SetButtonState(btn, true, position, 2);
    SetButtonState(btn, false, position, 2);
}

void DoubleClick(Button btn)
{
    CGPoint position = GetMousePosition();
    MouseDoubleClick(btn, position);
}

void MouseDrag(CGPoint position)
{
    DispatchMouseEvent(kCGMouseButtonLeft, kCGEventLeftMouseDragged, position, 1, 0);
}

void MouseStartDrag(CGPoint position)
{
    SetButtonState(kButtonLeft, true, position, 1);
    MouseDrag(position);
}

void MouseEndDrag(CGPoint position)
{
    MouseDrag(position);
    SetButtonState(kButtonLeft, false, position, 1);
}


#if __cplusplus
}
#endif

